In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pathlib import Path

from attila.data.parse import parse_data, get_data
from attila.util.config import get_config
from attila.util.plots import plot_sample
from attila.experiments.data import load_experiments
from attila.experiments.do import do_batch_experiments
from attila.experiments.tools import create_tex_experiments

In [3]:
_here = Path('.').resolve()
config = get_config(_here / 'config.ini')

data_path = _here / config.get('data', 'folder')
data_path = data_path.resolve()

out_path = Path(config.get('experiments', 'output folder')).resolve()
out_path.mkdir(parents=True, exist_ok=True)

images_path = data_path / config.get('data', 'images')
masks_path = data_path / config.get('data', 'masks')

In [4]:
raw = get_data(images_path, masks_path)
X, y = parse_data(
    raw,
    (config.getint('image', 'width'), config.getint('image', 'height'))
)

In [5]:
experiments_file = _here / 'experiments.json'
experiments = load_experiments(experiments_file)

In [ ]:
do_batch_experiments(experiments, (X, y), config, out_path)

testing data: X ~ (28, 512, 512, 1), y ~ (28, 512, 512, 3)
ready to perform #1 / 1 batch of experiments
augmented training data: X ~ (216, 512, 512, 1), y ~ (216, 512, 512, 3)
ready to perform 8 experiments
=== experiment #1 / 8: with_same
= dataset training: X ~ (216, 512, 512, 1), y ~ (216, 512, 512, 3)
= dataset validation: X ~ (10, 512, 512, 1), y ~ (10, 512, 512, 3)
= dataset test: X ~ (28, 512, 512, 1), y ~ (28, 512, 512, 3)
=== model
= # layers: 45
= # total params: 31377923
= # trainable params: 31377923
= # non-trainable params: 0
Epoch 1/2
54/54 [==============================] - 18s 327ms/step - loss: 2.2247 - accuracy: 0.8807 - batch_metric-mean_IoU: 0.6721 - batch_metric-DSC: 0.7877 - val_loss: 0.1538 - val_accuracy: 0.9511 - val_batch_metric-mean_IoU: 0.6062 - val_batch_metric-DSC: 0.7471 - lr: 0.0010
Epoch 2/2
7/7 [==============================] - 1s 84ms/step
=== evaluation stats
= metrics on test set (size: 28)
= batch_metric-mean_IoU ~ mean 0.691 median 0.664 std 0.1

In [ ]:
create_tex_experiments(config, out_path, out_path / 'tables.tex')